In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [2]:
# 获取数据集
dataset = load_wine()

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.25, random_state=0)

# 特征工程标准化
scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.fit_transform(X_test)

# 使用预估计器进行预估
estimator = SVC(kernel='linear', random_state=0)
estimator.fit(x_train, y_train)

SVC(kernel='linear', random_state=0)

In [3]:
# 开始模型评估，对比真实值
y_predict = estimator.predict(x_test)
print('y_predict:\n', y_predict)


y_predict:
 [0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [6]:
print('直接对比真实值与预测值:\n', y_test == y_predict)

直接对比真实值与预测值:
 [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]


In [5]:
# 计算准确率
score = estimator.score(x_test, y_test)
print('准确率为：\n', score)


准确率为：
 0.9777777777777777


In [7]:
# 打印分类报告
print('分类报告为： \n', classification_report(y_test, y_predict))

分类报告为： 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.95      0.98        21
           2       0.89      1.00      0.94         8

    accuracy                           0.98        45
   macro avg       0.96      0.98      0.97        45
weighted avg       0.98      0.98      0.98        45

